# Introduction to Motion Energy Models: Exercises

## Overview

In this exercise notebook, you'll apply the key concepts from the introduction to motion energy models. These exercises will help you develop intuition for how motion is represented in space-time and understand the foundational concepts that we'll build upon throughout the course.

### Learning Objectives
By completing these exercises, you will be able to:
- Visualize and analyze motion patterns in space-time
- Understand the components of a basic motion energy model
- Explore how different parameters affect motion representation
- Connect theoretical concepts to practical implementations

## Setup

Let's import the libraries we'll need for these exercises.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from mpl_toolkits.mplot3d import Axes3D
import scipy.signal as signal
import sys

# Add the utils package to the path
sys.path.append('../../..')
try:
    from motionenergy.utils import stimuli_generation, visualization
except ImportError:
    print("Note: utils modules not found. This is expected if you haven't implemented them yet.")

# For interactive plots
%matplotlib inline
from IPython.display import HTML, display

# Set plotting style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

## Exercise 1: Analyzing Motion in Space-Time

One of the key insights from the introduction is that motion can be represented as oriented patterns in space-time. In this exercise, you'll explore this concept by analyzing and creating space-time representations of motion.

### 1.1 Understanding Space-Time Plots

Here's a space-time plot of a simple moving bar:

In [ ]:
# Create a space-time plot of a moving bar
width = 100
frames = 100
bar_width = 5
speed = 1

# Create an empty space-time array
spacetime = np.zeros((frames, width))

# Add a moving bar
for t in range(frames):
    pos = (t * speed) % width
    bar_start = int(max(0, pos - bar_width/2))
    bar_end = int(min(width, pos + bar_width/2))
    spacetime[t, bar_start:bar_end] = 1

# Plot the space-time diagram
plt.figure(figsize=(10, 8))
plt.imshow(spacetime, cmap='gray', aspect='auto', origin='upper')
plt.title('Space-Time Representation of a Moving Bar')
plt.xlabel('Space (x)')
plt.ylabel('Time (t)')
plt.colorbar(label='Intensity')
plt.show()

**Questions:**

1. What does the orientation of the bar in the space-time plot tell you about the direction of motion?
2. How would the plot change if the bar moved in the opposite direction?
3. What would happen to the slope of the line if the bar moved faster? Slower?

**Your answers here:**
1. 
2. 
3. 

### 1.2 Creating Space-Time Representations

Now, let's create space-time representations for different types of motion. First, you'll need to implement a function to create a moving bar with variable speed and direction.

In [ ]:
def create_moving_bar_stimulus(width, height, frames, speed, direction='right'):
    """
    Create a stimulus with a bar moving at the specified speed and direction.
    
    Parameters:
    -----------
    width : int
        Width of the stimulus frame
    height : int
        Height of the stimulus frame
    frames : int
        Number of frames in the sequence
    speed : float
        Speed of the bar in pixels per frame
    direction : str
        Direction of motion ('right', 'left', 'up', or 'down')
        
    Returns:
    --------
    stimulus : ndarray
        3D array with dimensions (frames, height, width)
    """
    # Initialize the stimulus array
    stimulus = np.zeros((frames, height, width))
    
    # TODO: Implement the creation of a moving bar stimulus.
    # The bar should move in the specified direction at the given speed.
    # For horizontal motion (right/left), the bar should span the full height.
    # For vertical motion (up/down), the bar should span the full width.
    # The bar width should be approximately 10% of the frame dimension it's moving along.
    
    # HINTS:
    # 1. For horizontal motion, you need to update the x-coordinate (column index) over frames
    # 2. For vertical motion, you need to update the y-coordinate (row index) over frames
    # 3. You'll need conditional logic based on the 'direction' parameter
    # 4. Remember to handle cases where the bar goes outside the frame
    
    # Your implementation here:
    
    return stimulus

Now, let's create a function to visualize the space-time representation of a stimulus:

In [ ]:
def visualize_spacetime(stimulus, slice_type='horizontal', slice_position=None):
    """
    Visualize a stimulus in the space-time domain.
    
    Parameters:
    -----------
    stimulus : ndarray
        3D array with dimensions (frames, height, width)
    slice_type : str
        Type of slice to extract ('horizontal' or 'vertical')
    slice_position : int
        Position at which to extract the slice (default: middle of frame)
        
    Returns:
    --------
    fig : matplotlib figure
        Figure with the space-time visualization
    """
    frames, height, width = stimulus.shape
    
    # Set default slice position if not provided
    if slice_position is None:
        if slice_type == 'horizontal':
            slice_position = height // 2
        else:  # vertical
            slice_position = width // 2
    
    # TODO: Extract the appropriate slice from the stimulus based on slice_type
    # and slice_position. For a horizontal slice, extract a slice along a row of pixels
    # across all frames. For a vertical slice, extract a slice along a column of pixels
    # across all frames.
    
    # HINTS:
    # 1. For a horizontal slice, you want to extract stimulus[:, slice_position, :]
    #    This gives you all frames, a specific y-position, and all x-positions
    # 2. For a vertical slice, you want to extract stimulus[:, :, slice_position]
    #    This gives you all frames, all y-positions, and a specific x-position
    # 3. Make sure the dimensions of your extracted slice match the expected spacetime_slice shape
    
    # Your implementation here:
    
    # Create a place-holder spacetime_slice (you should replace this in your implementation)
    if slice_type == 'horizontal':
        spacetime_slice = np.zeros((frames, width))
    else:  # vertical
        spacetime_slice = np.zeros((frames, height))
    
    # Create the figure
    fig, ax = plt.subplots(figsize=(10, 8))
    im = ax.imshow(spacetime_slice, cmap='gray', aspect='auto', origin='upper')
    
    if slice_type == 'horizontal':
        ax.set_title(f'Horizontal Space-Time Slice (y={slice_position})')
        ax.set_xlabel('Space (x)')
    else:  # vertical
        ax.set_title(f'Vertical Space-Time Slice (x={slice_position})')
        ax.set_xlabel('Space (y)')
    
    ax.set_ylabel('Time (t)')
    plt.colorbar(im, ax=ax, label='Intensity')
    
    return fig

Now, use your functions to create and visualize stimuli with different motion patterns:

In [ ]:
# Create and visualize a rightward moving bar
width, height = 100, 100
frames = 50
speed = 2

# TODO: Create a stimulus using your create_moving_bar_stimulus function
# with direction='right'

# TODO: Visualize the stimulus in the space-time domain using your
# visualize_spacetime function with slice_type='horizontal'

In [ ]:
# TODO: Create and visualize a leftward moving bar
# Create a stimulus with direction='left'
# Visualize the space-time representation

In [ ]:
# TODO: Create and visualize an upward moving bar
# Create a stimulus with direction='up'
# Visualize the space-time representation with slice_type='vertical'

In [ ]:
# TODO: Create and visualize a downward moving bar
# Create a stimulus with direction='down'
# Visualize the space-time representation with slice_type='vertical'

### 1.3 Quantifying Motion from Space-Time

Now, let's implement a function to estimate the direction and speed of motion from a space-time representation. One approach is to measure the orientation of the pattern in the space-time domain.

In [ ]:
def estimate_motion_from_spacetime(spacetime_slice):
    """
    Estimate the direction and speed of motion from a space-time slice.
    
    Parameters:
    -----------
    spacetime_slice : ndarray
        2D array representing a slice through space-time
        
    Returns:
    --------
    direction : str
        Estimated direction of motion ('right', 'left', 'up', 'down', or 'unknown')
    speed : float
        Estimated speed in pixels per frame
    """
    # TODO: Implement an algorithm to estimate motion direction and speed
    # from the space-time slice. One approach is to analyze the orientation
    # of patterns in the slice.
    # 
    # TUTORIAL:
    # Motion in space-time creates slanted lines. The orientation of these lines tells us about motion:
    # - For horizontal slices:
    #   * Rightward motion creates lines slanted down and to the right
    #   * Leftward motion creates lines slanted down and to the left
    # - For vertical slices:
    #   * Downward motion creates lines slanted down and to the right
    #   * Upward motion creates lines slanted down and to the left
    #
    # The speed is related to the slope of these lines:
    # - Steeper slopes = slower motion
    # - Shallower slopes = faster motion
    # - Speed = dx/dt = 1/slope
    #
    # Techniques you might use:
    # 1. Hough Transform (from skimage.transform import hough_line, hough_line_peaks)
    #    This can detect lines and their angles in the space-time slice
    # 2. Gradient-based approaches (np.gradient)
    #    Calculate spatial and temporal gradients and analyze their ratio
    # 3. Edge detection followed by line fitting
    #    Detect edges then fit lines to estimate orientation
    
    # Your implementation here:
    
    # Placeholder return values (replace with your estimates)
    direction = 'unknown'
    speed = 0.0
    
    return direction, speed

Try your motion estimation function on the space-time slices you created:

In [ ]:
# TODO: Extract a space-time slice from one of your stimuli and
# apply your estimate_motion_from_spacetime function to estimate
# the direction and speed. Compare the results to the actual values
# you used when creating the stimulus.

## Exercise 2: Components of Motion Energy Models

Motion energy models consist of several key components that work together to detect motion. In this exercise, you'll explore these components and how they relate to each other.

### 2.1 Motion Energy Model Components

Below is a high-level diagram of a motion energy model:

In [ ]:
from matplotlib.patches import Rectangle, Arrow, FancyArrowPatch

def draw_model_diagram():
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Set up the axes
    ax.set_xlim(0, 10)
    ax.set_ylim(0, 6)
    ax.axis('off')
    
    # Draw the input
    ax.add_patch(Rectangle((0.5, 2.5), 1, 1, fill=True, alpha=0.3, color='blue'))
    ax.text(1, 2, "Visual\nInput", ha='center')
    
    # Draw the filters
    ax.add_patch(Rectangle((3, 4), 1.5, 1, fill=True, alpha=0.3, color='green'))
    ax.text(3.75, 3.7, "Even Filter\n(cosine phase)", ha='center')
    
    ax.add_patch(Rectangle((3, 1), 1.5, 1, fill=True, alpha=0.3, color='green'))
    ax.text(3.75, 0.7, "Odd Filter\n(sine phase)", ha='center')
    
    # Draw the squared outputs
    ax.add_patch(Rectangle((6, 4), 1, 1, fill=True, alpha=0.3, color='orange'))
    ax.text(6.5, 3.7, "Square", ha='center')
    
    ax.add_patch(Rectangle((6, 1), 1, 1, fill=True, alpha=0.3, color='orange'))
    ax.text(6.5, 0.7, "Square", ha='center')
    
    # Draw the sum
    ax.add_patch(Rectangle((8.5, 2.5), 1, 1, fill=True, alpha=0.3, color='red'))
    ax.text(9, 2, "Sum\n(Energy)", ha='center')
    
    # Draw the arrows
    ax.add_patch(FancyArrowPatch((1.5, 3), (3, 4.5), arrowstyle='->', mutation_scale=20, color='black'))
    ax.add_patch(FancyArrowPatch((1.5, 3), (3, 1.5), arrowstyle='->', mutation_scale=20, color='black'))
    
    ax.add_patch(FancyArrowPatch((4.5, 4.5), (6, 4.5), arrowstyle='->', mutation_scale=20, color='black'))
    ax.add_patch(FancyArrowPatch((4.5, 1.5), (6, 1.5), arrowstyle='->', mutation_scale=20, color='black'))
    
    ax.add_patch(FancyArrowPatch((7, 4.5), (8.5, 3), arrowstyle='->', mutation_scale=20, color='black'))
    ax.add_patch(FancyArrowPatch((7, 1.5), (8.5, 3), arrowstyle='->', mutation_scale=20, color='black'))
    
    ax.set_title("Basic Motion Energy Model", fontsize=16)
    
    return fig

draw_model_diagram()

**Questions:**

1. What is the purpose of having both even (cosine phase) and odd (sine phase) filters?
2. Why do we square the outputs of the filters?
3. What does the final summed output (energy) represent?
4. How would you modify this model to detect motion in the opposite direction?

**Your answers here:**
1. 
2. 
3. 
4. 

### 2.2 Simple Filter Responses

Let's implement a simplified version of the first stage of a motion energy model: applying spatiotemporal filters to a stimulus. We'll start with 1D space + time for simplicity.

In [ ]:
def create_1d_spatiotemporal_filters(space_size, time_size, speed, direction=1):
    """
    Create a pair of 1D+time spatiotemporal filters (even and odd phase).
    
    Parameters:
    -----------
    space_size : int
        Size of the spatial dimension
    time_size : int
        Size of the temporal dimension
    speed : float
        Speed of the filter's preferred motion
    direction : int
        Direction of the filter's preferred motion (1 for rightward, -1 for leftward)
        
    Returns:
    --------
    even_filter : ndarray
        2D array with dimensions (time_size, space_size) for the even phase filter
    odd_filter : ndarray
        2D array with dimensions (time_size, space_size) for the odd phase filter
    """
    # TODO: Implement the creation of a pair of 1D+time spatiotemporal filters.
    # These filters should be selective for motion at the specified speed and direction.
    # The even filter should have cosine phase, and the odd filter should have sine phase.
    # 
    # TUTORIAL:
    # 1. Understand space-time orientation:
    #    - A motion-selective filter is oriented in space-time in the direction of motion
    #    - The slope of this orientation is determined by the speed
    #    - direction=1 means rightward motion (positive slope), direction=-1 means leftward (negative slope)
    # 
    # 2. Creating a spatiotemporal Gabor filter:
    #    a) Create a 2D coordinate system (x, t) covering your space-time dimensions
    #    b) Create a rotated coordinate system (x', t') that's oriented along the motion trajectory
    #       x' = x*cos(θ) + t*sin(θ)  # Where θ depends on speed and direction
    #       t' = -x*sin(θ) + t*cos(θ)
    #    c) Create a Gabor filter using these rotated coordinates:
    #       even_filter = exp(-(x'^2 + t'^2)/(2*sigma^2)) * cos(2π*f*x')
    #       odd_filter = exp(-(x'^2 + t'^2)/(2*sigma^2)) * sin(2π*f*x')
    #    d) The spatial frequency f and Gaussian envelope sigma determine the filter's selectivity
    #
    # 3. Parameters to consider:
    #    - Spatial frequency (f): Higher values make the filter selective for higher spatial frequencies
    #    - Sigma: Controls the size of the Gaussian envelope
    #    - Orientation angle (θ): Determined by speed and direction, θ = arctan(direction * speed)
    
    # Your implementation here:
    
    # Placeholder return values (replace with your filters)
    even_filter = np.zeros((time_size, space_size))
    odd_filter = np.zeros((time_size, space_size))
    
    return even_filter, odd_filter

In [ ]:
def apply_energy_model(stimulus, even_filter, odd_filter):
    """
    Apply a motion energy model to a stimulus.
    
    Parameters:
    -----------
    stimulus : ndarray
        2D array with dimensions (time, space) representing the stimulus
    even_filter : ndarray
        2D array with dimensions (time, space) for the even phase filter
    odd_filter : ndarray
        2D array with dimensions (time, space) for the odd phase filter
        
    Returns:
    --------
    energy : float
        Motion energy response
    """
    # TODO: Implement the motion energy model:
    # 1. Apply the even filter to the stimulus using convolution
    # 2. Apply the odd filter to the stimulus using convolution
    # 3. Square both filter responses
    # 4. Sum the squared responses to get the motion energy
    
    # TUTORIAL:
    # The motion energy computation follows these steps:
    #
    # 1. Convolve the stimulus with each filter:
    #    - even_response = scipy.signal.convolve2d(stimulus, even_filter, mode='valid')
    #    - odd_response = scipy.signal.convolve2d(stimulus, odd_filter, mode='valid')
    #    - The 'valid' mode means only return output where filters and input fully overlap
    #
    # 2. Square the responses:
    #    - even_response_squared = even_response ** 2
    #    - odd_response_squared = odd_response ** 2
    #
    # 3. Sum the squared responses:
    #    - energy = np.sum(even_response_squared + odd_response_squared)
    #      or for a more detailed output:
    #    - energy_map = even_response_squared + odd_response_squared
    #
    # 4. IMPORTANT: The squaring and summing steps make the energy model phase-invariant,
    #    which means it responds similarly regardless of the exact position of features
    #    within the receptive field
    
    # Your implementation here:
    
    # Placeholder return value (replace with your calculation)
    energy = 0.0
    
    return energy

Let's create a simple 1D+time stimulus and test your implementation:

In [ ]:
def create_1d_moving_bar(space_size, time_size, bar_width, speed, direction=1):
    """
    Create a 1D bar moving over time.
    
    Parameters:
    -----------
    space_size : int
        Size of the spatial dimension
    time_size : int
        Number of time frames
    bar_width : int
        Width of the bar in pixels
    speed : float
        Speed of the bar in pixels per frame
    direction : int
        Direction of motion (1 for rightward, -1 for leftward)
        
    Returns:
    --------
    stimulus : ndarray
        2D array with dimensions (time_size, space_size)
    """
    stimulus = np.zeros((time_size, space_size))
    
    for t in range(time_size):
        center = int((space_size / 2) + direction * speed * (t - time_size / 2))
        start = max(0, center - bar_width // 2)
        end = min(space_size, center + bar_width // 2)
        if start < space_size and end > 0:
            stimulus[t, start:end] = 1.0
            
    return stimulus

# Create a simple rightward moving bar
space_size = 50
time_size = 30
bar_width = 5
speed = 1
direction = 1  # rightward

stimulus_right = create_1d_moving_bar(space_size, time_size, bar_width, speed, direction)

# Visualize the stimulus
plt.figure(figsize=(10, 6))
plt.imshow(stimulus_right, cmap='gray', aspect='auto', origin='upper')
plt.title(f'1D Space-Time: {"Rightward" if direction==1 else "Leftward"} Moving Bar')
plt.xlabel('Space')
plt.ylabel('Time')
plt.colorbar(label='Intensity')
plt.show()

In [ ]:
# Create filters for rightward motion
even_filter_right, odd_filter_right = create_1d_spatiotemporal_filters(space_size, time_size, speed, direction=1)

# Create filters for leftward motion
even_filter_left, odd_filter_left = create_1d_spatiotemporal_filters(space_size, time_size, speed, direction=-1)

# Visualize the filters
fig, axs = plt.subplots(2, 2, figsize=(12, 8))

axs[0, 0].imshow(even_filter_right, cmap='RdBu_r', aspect='auto', origin='upper')
axs[0, 0].set_title('Even Filter (Rightward)')
axs[0, 0].set_xlabel('Space')
axs[0, 0].set_ylabel('Time')

axs[0, 1].imshow(odd_filter_right, cmap='RdBu_r', aspect='auto', origin='upper')
axs[0, 1].set_title('Odd Filter (Rightward)')
axs[0, 1].set_xlabel('Space')
axs[0, 1].set_ylabel('Time')

axs[1, 0].imshow(even_filter_left, cmap='RdBu_r', aspect='auto', origin='upper')
axs[1, 0].set_title('Even Filter (Leftward)')
axs[1, 0].set_xlabel('Space')
axs[1, 0].set_ylabel('Time')

axs[1, 1].imshow(odd_filter_left, cmap='RdBu_r', aspect='auto', origin='upper')
axs[1, 1].set_title('Odd Filter (Leftward)')
axs[1, 1].set_xlabel('Space')
axs[1, 1].set_ylabel('Time')

plt.tight_layout()
plt.show()

In [ ]:
# TODO: Now test your energy model on stimuli moving in both directions
# (rightward and leftward) and with filters tuned to both directions.
# Calculate and compare the energy responses for each combination.

# Create leftward moving stimulus
stimulus_left = create_1d_moving_bar(space_size, time_size, bar_width, speed, direction=-1)

# Apply your energy model to each combination of stimulus and filter
# Calculate energy responses

# Visualize the results

### 2.3 Direction Selectivity

The motion energy model should respond more strongly to motion in its preferred direction. Let's explore this direction selectivity.

In [ ]:
def compute_direction_selectivity(stimulus, preferred_speed, angles):
    """
    Compute directional tuning curve for a given stimulus.
    
    Parameters:
    -----------
    stimulus : ndarray
        2D array with dimensions (time, space) representing the stimulus
    preferred_speed : float
        Preferred speed for the filters
    angles : array-like
        Array of angles (in degrees) to test
        
    Returns:
    --------
    responses : ndarray
        Array of energy responses for each angle
    """
    # TODO: Implement a function that computes the response of energy models
    # tuned to different directions (specified by angles) for a given stimulus.
    # 
    # TUTORIAL:
    # For a 1D space + time stimulus, angles of 0° and 180° represent rightward and leftward motion.
    # For each angle:
    # 1. Convert the angle to a direction vector:
    #    - direction = np.cos(angle_radians)  # 1 for rightward (0°), -1 for leftward (180°)
    # 
    # 2. Create filters tuned to that direction:
    #    - even_filter, odd_filter = create_1d_spatiotemporal_filters(
    #        space_size, time_size, preferred_speed, direction)
    # 
    # 3. Apply the energy model to get a response:
    #    - energy = apply_energy_model(stimulus, even_filter, odd_filter)
    # 
    # 4. Store the response for each angle:
    #    - responses[i] = energy
    #
    # Note: For a comprehensive directional tuning curve, test angles spanning the full 360°,
    # even though in 1D+time we're primarily interested in 0° (right) and 180° (left).
    # This prepares us for 2D+time in later exercises.
    
    # Your implementation here:
    
    # Placeholder return value (replace with your calculations)
    responses = np.zeros(len(angles))
    
    return responses

In [ ]:
# TODO: Use your compute_direction_selectivity function to generate
# tuning curves for rightward and leftward moving stimuli.
# Plot the results on polar coordinates to visualize the directional tuning.

# Define angles to test
angles = np.linspace(0, 360, 16, endpoint=False)

# Compute direction selectivity

# Plot the results

## Exercise 3: Motion Energy Parameters

In this exercise, you'll explore how different parameters affect motion energy models. You'll implement a simple interface to visualize these effects.

In [ ]:
from ipywidgets import interact, FloatSlider, IntSlider, Dropdown

def explore_filter_parameters(spatial_freq=0.1, temporal_freq=0.1, direction=0, sigma_space=10, sigma_time=10):
    """
    Visualize spatiotemporal filters with adjustable parameters.
    
    Parameters:
    -----------
    spatial_freq : float
        Spatial frequency of the filter
    temporal_freq : float
        Temporal frequency of the filter
    direction : float
        Direction of motion (in degrees)
    sigma_space : float
        Spatial spread of the Gaussian envelope
    sigma_time : float
        Temporal spread of the Gaussian envelope
    """
    # TODO: Implement a function that creates and visualizes spatiotemporal
    # filters with the specified parameters. The visualization should include:
    # - The even and odd filters
    # - How they respond to example stimuli
    # - Motion energy calculations
    
    # Your implementation here:
    
    # Placeholder visualization
    fig, axs = plt.subplots(1, 2, figsize=(12, 5))
    axs[0].text(0.5, 0.5, "Even Filter\nwould be shown here", ha='center', va='center')
    axs[0].set_title("Even Filter")
    axs[0].set_xlim(0, 1)
    axs[0].set_ylim(0, 1)
    
    axs[1].text(0.5, 0.5, "Odd Filter\nwould be shown here", ha='center', va='center')
    axs[1].set_title("Odd Filter")
    axs[1].set_xlim(0, 1)
    axs[1].set_ylim(0, 1)
    
    plt.suptitle(f"Spatiotemporal Filters (SF={spatial_freq}, TF={temporal_freq}, Dir={direction}°)")
    plt.tight_layout()
    plt.show()

# Create an interactive widget to explore filter parameters
interact(
    explore_filter_parameters,
    spatial_freq=FloatSlider(min=0.01, max=0.5, step=0.01, value=0.1, description="Spatial Freq:"),
    temporal_freq=FloatSlider(min=0.01, max=0.5, step=0.01, value=0.1, description="Temporal Freq:"),
    direction=FloatSlider(min=0, max=360, step=15, value=0, description="Direction (°):"),
    sigma_space=IntSlider(min=1, max=20, value=10, description="Spatial σ:"),
    sigma_time=IntSlider(min=1, max=20, value=10, description="Temporal σ:")
);

**Questions:**

1. How does changing the spatial frequency affect the filter's selectivity for different speeds?
2. How does changing the temporal frequency affect the filter's selectivity for different speeds?
3. What is the relationship between spatial frequency, temporal frequency, and preferred speed?
4. How do the sigma parameters affect the filter's selectivity?

**Your answers here:**
1. 
2. 
3. 
4. 

## Summary

In these exercises, you've explored the fundamental concepts of motion energy models:

1. You've analyzed and created space-time representations of motion, which are central to understanding how motion can be detected from a sequence of static images.

2. You've implemented components of a basic motion energy model, including spatiotemporal filters and the energy calculation, and explored how these components work together to detect motion.

3. You've investigated how different parameters affect the behavior of motion energy models, building intuition for how these models can be tuned to detect different types of motion.

In the next section, we'll explore visual stimuli in more detail, which will allow us to create and manipulate more complex motion patterns.